## imports



In [ ]:
from os.path import join
from json import loads
from collections import defaultdict

## environment variables

In [ ]:
DATABASE_NAME = join('.', "farmers-protest-tweets-2021-03-5.json")

## classes

In [ ]:
class RefInt:
    def __init__(self, reference):
        self.ref = reference
    def __lt__(self, other):
        return self.ref < other.ref

# FEATURE CODE

 ## feature 1 code

In [ ]:
def top10_retweets():
    top10 = list()
    ints = dict()
    def hook(dict_):
        if type(dict_) != dict:
            return dict_
        if "url" in dict_ and "retweetCount" in dict_:
            url = dict_["url"]
            retwcount = dict_["retweetCount"]
            if url not in ints:
                ints[url] = RefInt(retwcount)
                top10.append((ints[url], url))
            elif ints[url].ref < retwcount:
                ints[url].ref = retwcount
                if (ints[url], url) not in top10:
                    top10.append((ints[url], url))
            else:
                return {}
            top10.sort(reverse=True)
            del top10[10:]
            return {}
    with open(DATABASE_NAME, "r", encoding="utf-8") as file:
        for line in file:
            loads(f'{line.strip()}',  object_hook=hook)
    return ("number   | url", tuple((n.ref, url) for n, url in top10))

## feature 2 code

In [ ]:
def top10_twit_senders():
    top10 = list()
    twits = defaultdict(set)
    def hook(dict_):
        if type(dict_) != dict:
            return dict_
        elif "id" in dict_ and "user" in dict_:
            ret = {
                "id": dict_["id"]
            }
            if "user" in dict_:
                ret.update({"user": dict_["user"]["id"]})
            return ret
        elif "id" in dict_:
            return {
                "id": dict_["id"]
            }
    with open(DATABASE_NAME, "r", encoding="utf-8") as file:
        for line in file.readlines():
            data = loads(f'{line.strip()}',  object_hook=hook)
            twits[data["user"]].add(data["id"])
    top = [(len(set_), user) for user, set_ in twits.items()]
    top.sort(reverse=True)
    return (("number   | user id"), top[:10])

## feature 3 code

In [ ]:
def top10_twit_days():
    top10 = list()
    twits = set()
    days = defaultdict(set)
    def hook(dict_):
        if type(dict_) != dict:
            return dict_
        elif "id" in dict_ and "created" in dict_:
            ret = {
                "id": dict_["id"]
            }
            if dict_["id"] in twits:
                return {}
            if "created" in dict_:
                if not dict_["created"]:
                    return {}
                ret.update({"created": dict_["created"].split("T")[0]})
            days[dict_["created"].split("T")[0]].add(dict_["id"])
            twits.add(dict_["id"])
            return ret
    with open(DATABASE_NAME, "r", encoding="utf-8") as file:
        for line in file.readlines():
            loads(f'{line.strip()}',  object_hook=hook)
    top = [(len(set_), user) for user, set_ in days.items()]
    top.sort(reverse=True)
    return (("number   | date"), top[:10])

## feature 4 code

In [ ]:
def top10_hashtags():
    top10 = list()
    hashtags = defaultdict(set)
    def hook(dict_):
        if type(dict_) != dict:
            return dict_
        elif "url" in dict_ and "content" in dict_:
            ret = {
                "id": dict_["id"]
            }
            if "content" in dict_:
                ret.update({"content": dict_["content"]})
            return ret
        elif "id" in dict_:
            return {
                "id": dict_["id"]
            }
    with open(DATABASE_NAME, "r", encoding="utf-8") as file:
        for line in file.readlines():
            data = loads(f'{line.strip()}',  object_hook=hook)
            twit_hashtags = list()
            current = ""
            in_hashtag = False
            for c in data["content"]:
                if "#" == c:
                    in_hashtag = True
                elif c == " ":
                    if in_hashtag:
                        twit_hashtags.append("#"+current)
                        current = ""
                    in_hashtag = False
                else:
                    if in_hashtag:
                        current += c
            if in_hashtag:
                twit_hashtags.append("#"+current)
            for hashtag in twit_hashtags:
                hashtags[hashtag.lower()].add(data["id"])
    top = [(len(set_), user) for user, set_ in hashtags.items()]
    top.sort(reverse=True)
    return (("number   | Hashtag"), top[:10])

## main function code

In [ ]:
def Main():
    functions = {"1": top10_retweets, "2": top10_twit_senders, "3": top10_twit_days, "4": top10_hashtags}
    while True:
        print("choose an action")
        print("[1] top 10 most retweeted tweets")
        print("[2] top 10 users with most tweets")
        print("[3] top 10 days with most tweets")
        print("[4] top 10 most used hashtags")
        print("[any other] Exit")
        input_ = input("$>").strip()
        if input_ not in functions:
            break
        out1, out2 = functions[input_]()
        print(out1)
        for out21, out22 in out2:
            print(f"{out21:8d} | {out22}")
    print("goodbye")

# Main function execution

In [ ]:
Main()